In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
samsung_support = pd.DataFrame({'title' : [], 'date': [], 'user':[], 'nb_views':[], 'nb_replies':[], 'link':[]})

for i in range(1,134):
    html = requests.get('https://us.community.samsung.com/t5/Galaxy-S8-Questions-and-Answers/bd-p/GS8QA/page/'+str(i)) # pour pouvoir itérer
    soup = BeautifulSoup(html.text, 'lxml')
    
    bs_object = soup.find_all('div', attrs = {"class": u"lia-component-messages-column-message-info"}) # Attention au u
    df = pd.DataFrame({'title' : [], 'date': [], 'user':[], 'nb_views':[], 'nb_replies':[], 'link':[]})

    for s,index in zip(bs_object,range(len(bs_object))): 
        
        df.loc[index,'title'] = s.find('div', attrs={"class":u"MessageSubjectCell"}).get_text().strip()
    
        try:
            df.loc[index,'date'] = s.find('span', attrs={"class":u"DateTime"}).find('span', attrs={"class":u"local-date"}).get_text().strip()
        except AttributeError:
            df.loc[index,'date'] = s.find('span', attrs={"class":u"DateTime"}).find('span', attrs={"class":u"local-friendly-date"}).get_text().strip()
    
        df.loc[index,'user'] = s.find('span', attrs={"class":u"UserName"}).get_text().strip()
        
        df.loc[index,'nb_views'] = s.find_all('span', attrs={"class":u"lia-message-stats-count"})[0].get_text().strip()
    
        df.loc[index,'nb_replies'] = s.find_all('span', attrs={"class":u"lia-message-stats-count"})[1].get_text().strip()
        
        df.loc[index,'link'] = s.find('div', attrs={"class":u"MessageSubject"}).find('a').get('href').strip() 
        
    samsung_support = samsung_support.append(df, ignore_index = True)

samsung_support.shape

(3991, 6)

In [3]:
samsung_support.head()

,date,link,nb_replies,nb_views,title,user
0,‎03-30-2017,/t5/Galaxy-S8-Questions-and-Answers/Galaxy-S8-...,8586,74,Galaxy S8/S8+ Thoughts & Info\n ...,SamsungMarcos
1,‎11-27-2017,/t5/Galaxy-S8-Questions-and-Answers/pop-ups-Ad...,217,4,pop ups/ Adds,Rach715
2,24m ago,/t5/Galaxy-S8-Questions-and-Answers/Can-t-make...,7,1,Can't make calls,userGKEADqAA59
3,‎07-13-2017,/t5/Galaxy-S8-Questions-and-Answers/S8-music-p...,11350,38,S8 music player stops on it's own.\n ...,userExQfppLD9o
4,‎06-16-2017,/t5/Galaxy-S8-Questions-and-Answers/Very-frust...,10750,21,Very frustrated with Samsung Premium Care\n ...,userGX0F6EIpsT


In [4]:
thread = pd.DataFrame(columns = ['thread'])
for i in samsung_support.index:
    try:
        thread.loc[i] = [str(samsung_support['link'][i].split('/td-p/')[1])] 
    except IndexError:
        thread.loc[i] = [str(samsung_support['link'][i].split('/m-p/')[1])]
        
samsung_support = pd.merge(samsung_support, thread, left_index = True, right_index = True)

samsung_support.head()

,date,link,nb_replies,nb_views,title,user,thread
0,‎03-30-2017,/t5/Galaxy-S8-Questions-and-Answers/Galaxy-S8-...,8586,74,Galaxy S8/S8+ Thoughts & Info\n ...,SamsungMarcos,83749
1,‎11-27-2017,/t5/Galaxy-S8-Questions-and-Answers/pop-ups-Ad...,217,4,pop ups/ Adds,Rach715,218465
2,24m ago,/t5/Galaxy-S8-Questions-and-Answers/Can-t-make...,7,1,Can't make calls,userGKEADqAA59,252246
3,‎07-13-2017,/t5/Galaxy-S8-Questions-and-Answers/S8-music-p...,11350,38,S8 music player stops on it's own.\n ...,userExQfppLD9o,136597
4,‎06-16-2017,/t5/Galaxy-S8-Questions-and-Answers/Very-frust...,10750,21,Very frustrated with Samsung Premium Care\n ...,userGX0F6EIpsT,121544


In [11]:
samsung_support.isnull().sum()

date          0
link          0
nb_replies    0
nb_views      0
title         0
user          0
thread        0
dtype: int64

In [15]:
samsung_support = samsung_support.replace(r'\n',' ', regex=True)
samsung_support.to_csv("samsung_support.csv")

In [ ]:
samsung_comments = pd.DataFrame({'thread': [], 'comment':[]})

html = requests.get('https://us.community.samsung.com'+str(samsung_support['link'][0])) # pour pouvoir itérer
soup = BeautifulSoup(html.text, 'lxml')

bs = soup.find_all('div', attrs = {'class':u''})

In [ ]:
samsung_comments = pd.DataFrame({'thread': [], 'comment':[]})

html = requests.get('https://us.community.samsung.com/t5/Galaxy-S8-Questions-and-Answers/Galaxy-S8-S8-Thoughts-amp-Info/td-p/83749') # pour pouvoir itérer
soup = BeautifulSoup(html.text, 'lxml')

bs = soup.find_all('div', attrs = {'class':u'lia-quilt-row lia-quilt-row-main'})

